# 기상청 자료 크롤링

    API 없을 때, 크롤링으로 자료를 가져온다.
    API 있으면 이렇게 정리되지 않은 자료를 가져올 필요없지

## 1. HTML 가져온 뒤 파싱

In [3]:
import requests                  # 웹 페이지의 HTML을 가져오는 모듈
from bs4 import BeautifulSoup    # HTML을 파싱하는 모듈
 
# 웹 페이지를 가져온 뒤 BeautifulSoup 객체로 만듦
response = requests.get('https://pythondojang.bitbucket.io/weather/observation/currentweather.html')
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# cnt + shift + '-' 하면 셀이 구분됨
# a 누르면 위로 셀이 생기고, b 누르면 밑으로 셀이 생김
# dd = x , 셀을 삭제함
# esc 누르면 셀 테두리가 파란색으로 바뀌는데(관리모드), 이때 m 누르면 마크다운 형태로 바뀜
# #쓰고 한칸 띄우면 마크다운 생김!

In [18]:
soup.head()

[<title>도시별 현재날씨 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청 </title>,
 <link href="http://www.kma.go.kr/favicon2.ico" rel="shortcut icon"/>,
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <link href="/share/css/import.css?20160530" rel="stylesheet" type="text/css"/>,
 <script src="/share/js/jquery-1.7.1.min.js" type="text/javascript"></script>,
 <script src="/share/js/common.js?ver=20150417" type="text/javascript"></script>,
 <link href="/share/css/warninglayer.css" rel="stylesheet" type="text/css"/>,
 <meta content="기상청 " name="title"/>,
 <meta content="기상청" name="author"/>,
 <meta content="날씨, 기상청" name="keywords"/>,
 <meta content="서울시 기상청 사이트입니다." name="description"/>,
 <meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>,
 <meta content="../../images/weather/observation/h4_1_1.gif" name="parent-image"/>,
 <meta content="지상관측자료" name="parent-alt"/>,
 <meta content="../../images/weather/observation/h5_1_1.gif" name="title-image"/>,
 <meta content="도시별 현재

In [6]:
table = soup.find('table', { 'class': 'table_develop3' })    # <table class="table_develop3">을 찾음
#fn + 12 => html창이 뜨는데, 'table'에서 class명이 'table_develop3'이라는 자료를 찾는거임!
#방금 한 과정처럼, 원하는 데이터를 끌어오는 것이 '크롤링'이다

In [20]:
# table

## 2. 필요한 부분을 리스트로 저장

In [21]:
data = []                            # 데이터를 저장할 리스트 생성
for tr in table.find_all('tr'):      # 모든 <tr> 태그를 찾아서 반복(각 지점의 데이터를 가져옴)
    tds = list(tr.find_all('td'))    # 모든 <td> 태그를 찾아서 리스트로 만듦 (각 날씨 값을 리스트로 만듦)
    for td in tds:                   # <td> 태그 리스트 반복(각 날씨 값을 가져옴)
        if td.find('a'):             # <td> 안에 <a> 태그가 있으면(지점인지 확인)
            point = td.find('a').text    # <a> 태그 안에서 지점을 가져옴
            temperature = tds[5].text    # <td> 태그 리스트의 여섯 번째(인덱스 5)에서 기온을 가져옴
            humidity = tds[9].text       # <td> 태그 리스트의 열 번째(인덱스 9)에서 습도를 가져옴
            data.append([point, temperature, humidity])    # data 리스트에 지점, 기온, 습도를 추가

In [22]:
data[0:3]

[['서울', '25.6', '30'], ['백령도', '18.4', '62'], ['인천', '20.8', '54']]

## 3. 데이터프레임 생성

In [23]:
import pandas as pd

df = pd.DataFrame(data)

In [24]:
df.head()

,0,1,2
0,서울,25.6,30
1,백령도,18.4,62
2,인천,20.8,54
3,수원,25.0,41
4,동두천,24.9,34


In [25]:
print(df.index.name)

None


In [26]:
df = pd.DataFrame(df.values[:,1:], index=list(df[0]), columns=['기온', '습도'])
#[a:b,c:d]
# a:b 행이고 c:d 열

In [27]:
df.head(2)

,기온,습도
서울,25.6,30
백령도,18.4,62


## 4. CSV 파일로 저장

In [28]:
with open('weather22.csv', 'w') as file:    # weather.csv 파일을 쓰기 모드로 열기 #a면 추가, w 덮어쓰기(있던거 다 지우고), //w 또는 a가 저장기능임(저장이 우리가 아는 저장이 아닌데..ㅎㅎ)
    file.write('point,temperature,humidity\n')                  # 컬럼 이름 추가
    for i in data:                                              # data를 반복하면서
        file.write('{0},{1},{2}\n'.format(i[0], i[1], i[2]))    # 지점,온도,습도를 줄 단위로 저장